In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import numpy as np
# csv data file from zip file download from 
# http://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivtagmonat.html
# Messdatum als index und Interpretation als Datum (nicht nur String) 
#df=pd.read_csv('http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/sw/source/produkt_klima_Tageswerte_19000801_20151231_05792.txt',';\s*',index_col='MESS_DATUM',parse_dates=['MESS_DATUM'],skipfooter=1)
#df=pd.read_csv('http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/sw/source/produkt_klima_tag_19000801_20181231_05792.txt',';\s*',index_col='MESS_DATUM',parse_dates=['MESS_DATUM'],engine='python')

df=pd.read_csv('http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/sw/source/energy-charts_Oeffentliche_Nettostromerzeugung_in_Deutschland_2021.csv',index_col='Datum (GMT+1)',parse_dates=['Datum (GMT+1)'],engine='python')
print (df.size)
print (df.columns)


In [ ]:
df['REother']=df['Laufwasser'] + df['Biomasse'] + df['Speicherwasser'] + df['Pumpspeicher'] + df['Geothermie']
df['LastRed']=df.Last-df.REother
df['Wind'] = df['Wind Onshore'] + df['Wind Offshore']

In [ ]:
df.describe()

In [ ]:
retypes = ['Laufwasser', 'Biomasse', 'Geothermie', 'Speicherwasser', 'Pumpspeicher', 'Andere Erneuerbare', 'Müll', 'Wind Offshore', 'Wind Onshore',
       'Solar']
reotot = df.REother.sum()
ltot = df.Last.sum()
for r in retypes+['Last']:
    rtot = df[r].sum()
    print(f"{r:20s} {rtot:10.0f}  {rtot/reotot:.3f}  {rtot/ltot:.3f}")

In [ ]:
df.Last.sum()/(4*1e6)

In [ ]:
days = '2021-08-05'
days2 ='2021-08-16'
dayw = '2021-02-05'
dayw2 ='2021-02-16'
# random winter period
df.Last[dayw:dayw2].plot()
df.Solar[dayw:dayw2].plot()
df.Wind[dayw:dayw2].plot()
df.REother[dayw:dayw2].plot()
plt.legend();
#(df.Solar[day:day2]*fsol*rsol+df.Wind[day:day2]*fwin*rwin).plot()
fig=plt.figure()
# random summer period
df.Last[days:days2].plot()
df.Solar[days:days2].plot()
df.Wind[days:days2].plot()
df.REother[days:days2].plot()
plt.legend();

In [ ]:
# weekly sum over year
dfm=df.resample('w').sum()
dfm.Last.plot()
dfm.Wind.plot()
dfm.Solar.plot()
dfm.REother.plot()
plt.legend();

#### Ratio last vs solar / wind


In [ ]:

tl=df.LastRed.sum()
tlt=df.Last.sum()

fsol = tl/df.Solar.sum()
fwin = tl/df.Wind.sum()
print(f"yearly sum ratios: Last/Solar: {fsol:.2f} Last/Wind: {fwin:.2f}")


In [ ]:
# determine gap between last and solar/wind
dfc = df[df.Solar*fsol<df.LastRed]
smiss=(dfc.LastRed-dfc.Solar*fsol).sum()
dfc = df[df.Wind*fwin<df.LastRed]
wmiss=(dfc.LastRed-dfc.Wind*fwin).sum()
print(f"yearly relative gap for scaled-up solar: {smiss/tl:.3f} and wind: {wmiss/tl:.3f}")

In [ ]:
tl/tlt

In [ ]:
# plot gap as function of solar/wind mix
rsa = np.linspace(0,0.5,20)
fa=np.zeros(20)
for i,rsol in enumerate(rsa):
    rwin = 1-rsol
    dfc = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin)<df.LastRed]
    emiss=(dfc.LastRed-(dfc.Solar*fsol*rsol+dfc.Wind*fwin*rwin)).sum()
    f=emiss/tl
    fa[i]=f
#    print(rsol,f)
plt.plot(rsa,fa)

In [ ]:
rsol = 0.22
rwin = 1-rsol
fsol*rsol,fwin*rwin

#### Fit to determine best solar/wind mix 

In [ ]:
def chi2(params):
    rsol = params[0]
    rwin = 1-rsol
    dfc = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin)<df.LastRed]
    emiss=(dfc.LastRed-(dfc.Solar*fsol*rsol+dfc.Wind*fwin*rwin)).sum()
    f=emiss/tlt
    return f


In [ ]:
from scipy.optimize import minimize
result = minimize(chi2, (0.5))
print(result)

---

**optimize solar/wind with constraint that excess energy = 3 * gap-energy**

In [ ]:
# constrain chi2 fit such that excess-energy = 3 times gap-energy using constraint option in fit

def chi2c(params):
    rsol,rf = params
    rwin = 1-rsol
    dfc = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin)*rf<df.LastRed]
    emiss=(dfc.LastRed-(dfc.Solar*fsol*rsol+dfc.Wind*fwin*rwin)*rf).sum()
    dfp = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin)*rf>df.LastRed]
    eplus=-(dfp.LastRed-(dfp.Solar*fsol*rsol+dfp.Wind*fwin*rwin)*rf).sum()
  
    f=emiss/tlt 
    return f
def constr(params):
    rsol,rf = params
    rwin = 1-rsol
    dfc = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin)*rf<df.LastRed]
    emiss=(dfc.LastRed-(dfc.Solar*fsol*rsol+dfc.Wind*fwin*rwin)*rf).sum()
    dfp = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin)*rf>df.LastRed]
    eplus=-(dfp.LastRed-(dfp.Solar*fsol*rsol+dfp.Wind*fwin*rwin)*rf).sum()
    
    
    f=emiss-eplus/3
    return f


In [ ]:
# see https://people.duke.edu/~ccc14/sta-663-2017/14C_Optimization_In_Python.html 
cons = ({'type': 'eq', 'fun' : constr})
result = minimize(chi2c, (0.5,1.), constraints=cons)

In [ ]:
result

In [ ]:
constr(result.x)

In [ ]:
rsol,rf = result.x
rwin = 1 - rsol
fsoltot = fsol * rsol * rf
fwintot = fwin * rwin * rf
print(f"Factor for solar {fsoltot:.2f}, Factor for wind {fwintot:.2f}") 

In [ ]:
solsum=df.Solar.sum()*fsoltot
winsum=df.Wind.sum()*fwintot
print(f" {solsum=:.1f} {winsum=:.1f} {tl=:.1f}")
print(f" ratio: {(solsum+winsum)/tl:.3f}")

In [ ]:
df['SWscaled']=df.Solar*fsoltot+df.Wind*fwintot
dfm=df.resample('w').sum()
dfm.LastRed.plot()
dfm.Wind.plot()
dfm.Solar.plot()
dfm.SWscaled.plot()

In [ ]:
days = '2021-08-05'
days2 ='2021-08-16'
dayw = '2021-02-05'
dayw2 ='2021-02-16'
# random winter period
df.LastRed[dayw:dayw2].plot()
(df.Solar[dayw:dayw2]*fsoltot).plot()
(df.Wind[dayw:dayw2]*fwintot).plot()
df.SWscaled[dayw:dayw2].plot()
plt.legend();
#(df.Solar[day:day2]*fsol*rsol+df.Wind[day:day2]*fwin*rwin).plot()
fig=plt.figure()
# random summer period
df.LastRed[days:days2].plot()
(df.Solar[days:days2]*fsoltot).plot()
(df.Wind[days:days2]*fwintot).plot()
df.SWscaled[days:days2].plot()
plt.legend();

In [ ]:
(df.SWscaled-df.LastRed).hist(bins=50)

In [ ]:
dfw=df['Solar'].resample('w').sum()
#dfw=df['Wind Offshore'].resample('w').sum()
dfw=df['Wind Onshore'].resample('w').sum()
dfw.plot();

In [ ]:
dfd=df.resample('d').sum()

In [ ]:
plt.plot(df['Wind Offshore'],df['Wind Onshore'],'.')

In [ ]:
plt.plot(dfd['Solar'],dfd['Wind'],'.')

In [ ]:
df.Last.sum(),df.Solar.sum(),df.Wind.sum()

In [ ]:
tl=df.Last.sum()
fsol = tl/df.Solar.sum()
fwin = tl/df.Wind.sum()
fsol,fwin


In [ ]:
dfc = df[df.Solar*fsol<df.Last]
smiss=(dfc.Last-dfc.Solar*fsol).sum()
dfc = df[df.Wind*fwin<df.Last]
wmiss=(dfc.Last-dfc.Wind*fwin).sum()
smiss/tl,wmiss/tl

In [ ]:

rsol = 0.3
rwin = 1-0.3
dfc = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin+df.Pumpspeicher)<df.Last]
emiss=(dfc.Last-(dfc.Solar*fsol*rsol+dfc.Wind*fwin*rwin+df.Pumpspeicher)).sum()
emiss/tl

In [ ]:
rsa = np.linspace(0,0.5,20)
fa=np.zeros(20)
for i,rsol in enumerate(rsa):
    rwin = 1-rsol
    dfc = df[(df.Solar*fsol*rsol+df.Wind*fwin*rwin+df.Pumpspeicher)<df.Last]
    emiss=(dfc.Last-(dfc.Solar*fsol*rsol+dfc.Wind*fwin*rwin+df.Pumpspeicher)).sum()
    f=emiss/tl
    fa[i]=f
#    print(rsol,f)
plt.plot(rsa,fa)

In [ ]:
smiss/tl

In [ ]:
len(dfc)

In [ ]:
len(df)

In [ ]:
1+1